# MAPD Demo: Multi-Agent Pickup and Delivery

This notebook demonstrates MAPD (Multi-Agent Pickup and Delivery) using the PIBT algorithm. We'll solve examples on both random and warehouse maps with visualizations.

## 1. Import Libraries

In [ ]:
from pypibt import MAPD, is_valid_mapd_solution

## 2. Load MAPD Instance

Load a MAPD problem with:
- Map file (grid layout)
- Task configuration (pickup/delivery tasks)
- Number of agents

In [ ]:
# Load problem instance
mapd = MAPD(
    "../assets/mapd/task-config.txt", 
    map_file="../assets/maps/random-32-32-10.map",
    num_agents=100,
    seed=42
)

print(f"MAPD Instance:")
print(f"  Grid size: {mapd.grid.shape}")
print(f"  Number of agents: {mapd.N}")
print(f"  Task frequency: {mapd.instance.task_frequency}")
print(f"  Total tasks: {mapd.instance.task_num}")

## 3. Solve the Problem

Run the MAPD solver until all tasks are completed or limits reached.

In [ ]:
# Solve MAPD
import time
start_time = time.perf_counter()
solution = mapd.run(max_timestep=1000)
comp_time = time.perf_counter() - start_time

print(f"Solution completed!")
print(f"Timesteps: {len(solution) - 1}")
print(f"Computation time: {comp_time:.2f}s")
print(f"Completed tasks: {len(mapd.completed_tasks)}/{mapd.instance.task_num}")

## 4. Validate Solution

In [ ]:
# Validate the solution
valid = is_valid_mapd_solution(mapd.grid, mapd.starts, solution, mapd.completed_tasks)
print(f"Valid solution: {valid}")

## 5. Save Solution

In [ ]:
# Save solution
output_file = "../assets/mapd/output_mapd.txt"
mapd.save_solution(output_file)
print(f"Solution saved to {output_file}")

## 6. Visualize Solution

Use the Python visualizer to create an animated video showing the agents executing pickup and delivery tasks.

In [ ]:
import sys
sys.path.append('..')

from scripts.visualize_mapd import MAPDVisualizer, load_solution, get_grid as viz_get_grid
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for notebook
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter

# Load the solution
grid_viz = viz_get_grid("../assets/maps/random-32-32-10.map")
solution_viz, agent_goals_viz = load_solution("../assets/mapd/output_mapd.txt")

print(f"Loaded solution: {len(solution_viz)} timesteps, {len(solution_viz[0])} agents")

# Create visualizer with interpolation for smooth animation
visualizer = MAPDVisualizer(grid_viz, solution_viz, agent_goals=agent_goals_viz, 
                            interpolation_steps=5, mode='mapd')

# Create animation and save as GIF (works without ffmpeg)
fig, anim = visualizer.animate(interval=20)

# Save as GIF using Pillow (no ffmpeg required)
anim.save("../assets/mapd/output_mapd.gif", writer=PillowWriter(fps=50))
plt.close(fig)

# Display the GIF in notebook
from IPython.display import Image
Image(filename="../assets/mapd/output_mapd.gif")

## 7. Warehouse Map Example

Now let's try a larger warehouse map with more agents.

In [ ]:
# Load warehouse map with MAPD configuration
warehouse_mapd = MAPD(
    "../assets/mapd/task-config.txt",
    map_file="../assets/maps/warehouse-20-40-10-2-2.map",
    num_agents=20,
    seed=42
)

# Solve
warehouse_solution = warehouse_mapd.run(max_timestep=2000)

# Validate
warehouse_valid = is_valid_mapd_solution(
    warehouse_mapd.grid, 
    warehouse_mapd.starts, 
    warehouse_solution, 
    warehouse_mapd.completed_tasks
)

print(f"Warehouse Results:")
print(f"  Grid: {warehouse_mapd.grid.shape}")
print(f"  Agents: {warehouse_mapd.N}")
print(f"  Valid: {warehouse_valid}")
print(f"  Timesteps: {len(warehouse_solution) - 1}")
print(f"  Completed tasks: {len(warehouse_mapd.completed_tasks)}/{warehouse_mapd.instance.task_num}")

## Summary

This notebook demonstrated:
- Loading MAPD instances with map and task configuration
- Solving with PIBT algorithm and dynamic task assignment
- Validating solutions
- Visualizing agent behavior

**Visualization Features:**
- **Smooth interpolation**: Agents move fluidly between timesteps
- **Agent status indicators**: 
  - Filled circles = carrying items (assigned)
  - Hollow circles = targeting pickup location
  - Transparent circles = free/idle agents
- **Dynamic goal lines**: Lines show each agent's current target (pickup or delivery)
- **GIF animations**: Animations saved and displayed as GIF images (no ffmpeg required)
- **Export options**: Animations automatically saved as GIF files

**Key Parameters:**
- `map_file`: Path to map file
- `task_config`: Path to task configuration file
- `num_agents`: Number of agents
- `seed`: Random seed for reproducibility
- `max_timestep`: Maximum simulation steps
- `interpolation_steps`: Frames per timestep for smooth animation (default: 5)
- `fps`: Frames per second for GIF export (default: 50)
- `mode`: Visualization mode - set to 'mapd' to show dynamic goals